In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import psycopg2
from datetime import datetime, timedelta, date
import shutil

parent_path = '/home/server/gli-data-science/akhiyar/data_req/used_release'
if os.path.exists(parent_path):
    print("exists")
    shutil.rmtree(parent_path)
os.makedirs(parent_path, exist_ok=True)
end_date = date.today()-timedelta(days=0)
start_date = end_date.replace(day=1)
print(start_date, end_date)

exists
2021-08-01 2021-08-27


In [2]:
def pro_connect():
    try:
        cpo = psycopg2.connect(host="35.240.137.10",
                                database="prd_order",
                                user="akhiyar_waladi",
                                password="nd4n6fk9")
        print("prd_order Connected!")
    except Exception as error:
        print("Error while connecting: ", error)
        sys.exit()
        
    return cpo

In [3]:
def pas_connect():
    try:
        cpp = psycopg2.connect(host="35.240.166.149",
                                database="prd_promo_stl_v2",
                                user="akhiyar_waladi",
                                password="nd4n6fk9")
        print("prd_promo Connected!")
    except Exception as error:
        print("Error while connecting: ", error)
        sys.exit()
        
    return cpp

In [4]:
## loop from start that we want
for start_date in pd.date_range(start_date, end_date, freq='D'):
    
    end_date = start_date + timedelta(days=1)
    
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    print(start_date_str, end_date_str)

    q_order = '''
    select tbto_id, tbto_no, tbtc_id, tbto_voucher_code, tbto_create_date, tbto_status
    from tb_transaction_order tto
    where tto.tbto_create_date between '{}' and '{}'
    and tbto_voucher_code != ''
    and tbto_status in ('12','14','15')

    '''.format(start_date_str, end_date_str)
    cpo = pro_connect()
    df_order = pd.read_sql(q_order, cpo)
    cpo.close()

    q_promo = '''
    select tbtvi_cart_id, tbtvi_registration_receipt, tbtvi_registration_no, tbtvi_created_date, tbtvi_updated_date, tbtvi_released 
    from tb_transaction_voucher_issued ttvi 
    where tbtvi_cart_id in {}
    '''.format(tuple(df_order['tbtc_id']))
    cpp = pas_connect()
    df_promo = pd.read_sql(q_promo, cpp)
    cpp.close()

    df_order_promo = pd.merge(df_order, df_promo, left_on='tbtc_id', right_on='tbtvi_cart_id')

    df_order_promo[df_order_promo['tbtvi_released'] == True].\
        to_csv('{}/{}.csv'.format(parent_path,start_date), index=False)

2021-08-01 2021-08-02
prd_order Connected!
prd_promo Connected!
2021-08-02 2021-08-03
prd_order Connected!
prd_promo Connected!
2021-08-03 2021-08-04
prd_order Connected!
prd_promo Connected!
2021-08-04 2021-08-05
prd_order Connected!
prd_promo Connected!
2021-08-05 2021-08-06
prd_order Connected!
prd_promo Connected!
2021-08-06 2021-08-07
prd_order Connected!
prd_promo Connected!
2021-08-07 2021-08-08
prd_order Connected!
prd_promo Connected!
2021-08-08 2021-08-09
prd_order Connected!
prd_promo Connected!
2021-08-09 2021-08-10
prd_order Connected!
prd_promo Connected!
2021-08-10 2021-08-11
prd_order Connected!
prd_promo Connected!
2021-08-11 2021-08-12
prd_order Connected!
prd_promo Connected!
2021-08-12 2021-08-13
prd_order Connected!
prd_promo Connected!
2021-08-13 2021-08-14
prd_order Connected!
prd_promo Connected!
2021-08-14 2021-08-15
prd_order Connected!
prd_promo Connected!
2021-08-15 2021-08-16
prd_order Connected!
prd_promo Connected!
2021-08-16 2021-08-17
prd_order Connecte

In [5]:
path = '/home/server/gli-data-science/akhiyar/data_req/used_release/' 
df = []
for f in os.listdir(path):
    df.append(pd.read_csv(os.path.join(path, f)))
df = pd.concat(df)

end_date = date.today()-timedelta(days=28)
start_date = end_date.replace(day=1)
spath = '{}/used_release_{}_{}.xlsx'\
            .format(parent_path, start_date.strftime("%d%b%y"),\
            end_date.strftime("%d%b%y")).format()

writer = pd.ExcelWriter(spath, engine='xlsxwriter') 
df.to_excel(writer, sheet_name='ReleasedVoucher', index=False, na_rep='NaN')

# Auto-adjust columns' width
for column in df:
    column_width = max(df[column].astype(str).map(len).max(), len(column))
    col_idx = df.columns.get_loc(column)
    writer.sheets['ReleasedVoucher'].set_column(col_idx, col_idx, column_width)

writer.save()

# df.to_excel('/home/server/gli-data-science/akhiyar/data_req/used_release/used_release_{}_{}.xlsx'\
#             .format(start_date.strftime("%d%b%y"),\
#             end_date.strftime("%d%b%y")).format(), index=False)